In [ ]:
account =  # your storage account name
container =  # your container name

bronze = f"abfss://{container}@{account}.dfs.core.windows.net/bronze"
silver = f"abfss://{container}@{account}.dfs.core.windows.net/silver"

In [ ]:
from pyspark.sql import functions as F

orders_bronze = f"{bronze}/orders"
fact_orders_silver = f"{silver}/fact_orders"

ob = spark.read.parquet(orders_bronze)

fo = (ob
    # normalize column names already done in bronze; cast types here
    .withColumn("order_id", F.col("order_id").cast("int"))
    .withColumn("user_id", F.col("user_id").cast("int"))
    .withColumn("eval_set", F.col("eval_set").cast("string"))
    .withColumn("order_number", F.col("order_number").cast("int"))
    .withColumn("order_dow", F.col("order_dow").cast("int"))
    # sample showed "08" / "07" — cast to int safely
    .withColumn("order_hour_of_day", F.col("order_hour_of_day").cast("int"))
    .withColumn("days_since_prior_order", F.col("days_since_prior_order").cast("double"))
    .select("order_id","user_id","eval_set","order_number","order_dow",
            "order_hour_of_day","days_since_prior_order")
)

# write silver
(fo.repartition(1)
   .write.mode("overwrite")
   .parquet(fact_orders_silver))

print("silver.fact_orders:", fact_orders_silver, "rows:", fo.count())

In [ ]:
products_bronze    = f"{bronze}/products"
aisles_bronze      = f"{bronze}/aisles"
departments_bronze = f"{bronze}/departments"
dim_products_silver = f"{silver}/dim_products"

p  = spark.read.parquet(products_bronze)
a  = spark.read.parquet(aisles_bronze)
d  = spark.read.parquet(departments_bronze)

# cast + join
p = (p
     .withColumn("product_id", F.col("product_id").cast("int"))
     .withColumn("product_name", F.col("product_name").cast("string"))
     .withColumn("aisle_id", F.col("aisle_id").cast("int"))
     .withColumn("department_id", F.col("department_id").cast("int"))
)

a = (a
     .withColumn("aisle_id", F.col("aisle_id").cast("int"))
     .withColumnRenamed("aisle", "aisle_name")
)

d = (d
     .withColumn("department_id", F.col("department_id").cast("int"))
     .withColumnRenamed("department", "department_name")
)

dim_products = (p
    .join(a, "aisle_id", "left")
    .join(d, "department_id", "left")
    .select("product_id","product_name","aisle_id","aisle_name","department_id","department_name")
)

(dim_products.repartition(1)
    .write.mode("overwrite")
    .parquet(dim_products_silver))

print(" silver.dim_products:", dim_products_silver, "rows:", dim_products.count())

In [ ]:
opp_bronze = f"{bronze}/order_products__prior"
opt_bronze = f"{bronze}/order_products__train"
fact_op_silver = f"{silver}/fact_order_products"

prior = (spark.read.parquet(opp_bronze)
         .withColumn("split_source", F.lit("prior")))

train = (spark.read.parquet(opt_bronze)
         .withColumn("split_source", F.lit("train")))

# ensure consistent types
def cast_op(df):
    return (df
        .withColumn("order_id", F.col("order_id").cast("int"))
        .withColumn("product_id", F.col("product_id").cast("int"))
        .withColumn("add_to_cart_order", F.col("add_to_cart_order").cast("int"))
        .withColumn("reordered", F.col("reordered").cast("int"))
        .select("order_id","product_id","add_to_cart_order","reordered","split_source")
    )

fact_op = cast_op(prior).unionByName(cast_op(train))

(fact_op.repartition(8)  # a few files for parallel reads later
   .write.mode("overwrite")
   .parquet(fact_op_silver))

print(" silver.fact_order_products:", fact_op_silver, "rows:", fact_op.count())

In [ ]:
fact_orders_silver = f"{silver}/fact_orders"
fact_op_silver     = f"{silver}/fact_order_products"
dim_users_silver   = f"{silver}/dim_users"
fup_silver         = f"{silver}/fact_user_product"

fo = spark.read.parquet(fact_orders_silver)
op = spark.read.parquet(fact_op_silver)

# basket sizes per order
basket_sizes = (op.groupBy("order_id").agg(F.count("*").alias("basket_size")))

# join to orders to compute user metrics
orders_plus = (fo.join(basket_sizes, "order_id", "left"))

# user reorder ratio: needs reordered flag joined at user level
user_reorders = (op.groupBy().count())  # just placeholder to avoid confusion

# dim_users: user-level rollups
dim_users = (orders_plus
  .groupBy("user_id")
  .agg(
      F.countDistinct("order_id").alias("total_orders"),
      F.avg("days_since_prior_order").alias("avg_days_between_orders"),
      F.expr("percentile_approx(order_hour_of_day, 0.5)").alias("median_order_hour"),
      F.expr("percentile_approx(order_dow, 0.5)").alias("median_order_dow"),
      F.avg("basket_size").alias("avg_basket_size")
  )
)

# compute user reorder ratio (share of line items with reordered=1)
user_reorder_ratio = (op.groupBy("order_id")
                        .agg(F.avg(F.col("reordered").cast("double")).alias("order_reorder_rate"))
                      .join(fo.select("order_id","user_id"), "order_id", "left")
                      .groupBy("user_id")
                      .agg(F.avg("order_reorder_rate").alias("reorder_ratio"))
                     )

dim_users = (dim_users.join(user_reorder_ratio, "user_id", "left")
                      .fillna({"reorder_ratio": 0.0})
)

(dim_users.repartition(1)
   .write.mode("overwrite")
   .parquet(dim_users_silver))

print("silver.dim_users:", dim_users_silver, "rows:", dim_users.count())

# (optional) user × product history for ML features and analytics
fup = (op.join(fo.select("order_id","user_id"), "order_id", "left")
         .groupBy("user_id","product_id")
         .agg(
             F.count("*").alias("times_purchased"),
             F.avg("add_to_cart_order").alias("avg_add_to_cart_position"),
             F.max("reordered").alias("ever_reordered")
         )
      )

(fup.repartition(8)
    .write.mode("overwrite")
    .parquet(fup_silver))

print("silver.fact_user_product:", fup_silver, "rows:", fup.count())